<a href="https://colab.research.google.com/github/dn-bit/Sentiment_analysis_on_imdb/blob/main/on_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("IMDB_Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
y=df["sentiment"]
y=pd.get_dummies(y,drop_first=True)
y.head()


,positive
0,1
1,1
2,1
3,0
4,1


In [4]:
from nltk.text import sent_tokenize
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tqdm

nltk.download('punkt')


nltk.download('wordnet')
lem=WordNetLemmatizer()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
corpus=[]
from tqdm import tqdm
nltk.download('stopwords')

for i in tqdm(range(0,len(df))):
  sent=re.sub('[^a-zA-Z]',' ',df["review"][i])
  sent=sent.lower()
  sent=sent.split()
  sent=[lem.lemmatize(word) for word in sent if not word in set(stopwords.words('english'))]
  sent=' '.join(sent)
  corpus.append(sent)





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 5526/5526 [03:27<00:00, 26.70it/s]


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
tf=CountVectorizer(binary=True)

x=tf.fit_transform(corpus).toarray()

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
import warnings
warnings.filterwarnings('ignore')

model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [10]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.87      0.80      0.83       614
           1       0.77      0.85      0.81       492

    accuracy                           0.82      1106
   macro avg       0.82      0.83      0.82      1106
weighted avg       0.83      0.82      0.82      1106



In [11]:
a="movie is average and very bad dialogue"

def clean_sent(sent):
  sent=re.sub('[^A-Za-z]',' ', sent)
  sent=sent.lower()
  sent=sent.split()
  sent=[lem.lemmatize(word) for word in sent if not word in set(stopwords.words('english'))]
  sent=' '.join(sent)
  return sent



a1=clean_sent(a)
a1=[a1]

x_new=tf.transform(a1).toarray()
#print(x_new)
predd_y=model.predict(x_new)

if predd_y==0:
  print("Negative")
else:
  print("Positive")

Negative


In [12]:
from sklearn.tree import DecisionTreeClassifier

model2=DecisionTreeClassifier()
model2.fit(x_train,y_train)
pred_y=model2.predict(x_test)

print(classification_report(y_test,pred_y))

              precision    recall  f1-score   support

           0       0.70      0.71      0.70       561
           1       0.70      0.69      0.69       545

    accuracy                           0.70      1106
   macro avg       0.70      0.70      0.70      1106
weighted avg       0.70      0.70      0.70      1106



In [13]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=100,)

rf.fit(x_train,y_train)
pred_y=rf.predict(x_test)

print(classification_report(y_test,pred_y))


              precision    recall  f1-score   support

           0       0.82      0.86      0.84       561
           1       0.85      0.81      0.83       545

    accuracy                           0.83      1106
   macro avg       0.83      0.83      0.83      1106
weighted avg       0.83      0.83      0.83      1106

